In [1]:
# Importer la bibliothèque pandas
import pandas as pd

# Charger le fichier Parquet
file_path = '/home/patea/code/Pateaanania/poipoi/resultat_concatenation_avec_date_heure.parquet'
df = pd.read_parquet(file_path)

# Filtrer pour garder uniquement les lignes où la colonne PATIENT contient des chiffres et exclure '99999999'
df_filtered = df[df['PATIENT'].apply(lambda x: str(x).isdigit())]
df_filtered = df_filtered[df_filtered['PATIENT'] != '99999999']

# Supprimer les colonnes inutiles, en conservant 'Localisation'
colonnes_a_supprimer = ['CHAMBRE', 'LIT', 'Date', 'Heure', 'LIT_DISPONIBLE', 'LIT_INDISPONIBLE', 
                        'EMPLACEMENT_VIDE', 'PATIENT_UNKNOWN']  # Localisation n'est pas dans cette liste
df_filtered = df_filtered.drop(columns=[col for col in colonnes_a_supprimer if col in df_filtered.columns])

# Renommer la colonne 'DateHeure' pour plus de clarté
df_filtered.rename(columns={'DateHeure': 'date'}, inplace=True)

# Assurer que la colonne 'date' est bien au format datetime
df_filtered['date'] = pd.to_datetime(df_filtered['date'], errors='coerce')

# Trier les données par patient et date
df_filtered = df_filtered.sort_values(by=['PATIENT', 'date'])

# Calculer l'écart de temps entre les enregistrements consécutifs pour chaque patient
df_filtered['Time_Diff'] = df_filtered.groupby('PATIENT')['date'].diff()

# Définir un seuil d'interruption (par exemple, 1 heure) pour identifier les séjours distincts
seuil_interruption = pd.Timedelta(hours=1)
df_filtered['New_Sejour'] = (df_filtered['Time_Diff'] > seuil_interruption).astype(int)

# Marquer le premier enregistrement comme début d'un séjour
df_filtered['New_Sejour'] = df_filtered.groupby('PATIENT')['New_Sejour'].cumsum()

# Regrouper pour définir les séjours distincts : obtenir la date d'entrée, de sortie et la localisation pour chaque séjour
sejours = df_filtered.groupby(['PATIENT', 'New_Sejour', 'Localisation']).agg(
    Date_Entree=('date', 'min'),
    Date_Sortie=('date', 'max')
).reset_index()

# Afficher les résultats des séjours distincts
print(sejours.head(10))


    PATIENT  New_Sejour Localisation         Date_Entree         Date_Sortie
0  10000153           0   CHA-3046-1 2024-01-16 07:00:00 2024-01-16 13:00:00
1  10000153           1   CHA-3046-1 2024-01-17 07:00:00 2024-01-17 13:00:00
2  10000153           2   CHA-3046-1 2024-01-18 07:00:00 2024-01-18 13:00:00
3  10000153           3   CHA-3046-1 2024-01-19 07:00:00 2024-01-19 13:00:00
4  10000153           4   CHA-3046-1 2024-01-20 07:00:00 2024-01-20 13:00:00
5  10000153           5   CHA-3046-1 2024-01-21 07:00:00 2024-01-21 13:00:00
6  10000179           0    HUR-012-1 2024-01-27 13:30:00 2024-01-28 15:00:00
7  10000179           0   NEU-2052-2 2024-01-28 15:30:00 2024-01-29 12:30:00
8   1000022           0     REA-D4-1 2024-01-16 07:00:00 2024-01-16 13:00:00
9   1000022           1     REA-D4-1 2024-01-17 07:00:00 2024-01-17 13:00:00


In [2]:
sejours.to_csv('sejours_patients.csv', index=False)
